In [1]:
!unzip Datasets

Archive:  Datasets.zip
replace Datasets/Test/audi/21.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [2]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt


2022-10-19 11:51:00.007520: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-19 11:51:00.258242: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-19 11:51:01.253155: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-10-19 11:51:01.253262: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [3]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = 'Datasets/train'
valid_path = 'Datasets/test'

In [4]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

resnet = ResNet50(input_shape=IMAGE_SIZE + [3], classes = 3, weights='imagenet', include_top=False)

2022-10-19 11:51:12.731150: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-19 11:51:12.769899: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-10-19 11:51:12.769917: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-10-19 11:51:12.770223: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

In [5]:

# don't train existing weights
for layer in resnet.layers:
    layer.trainable = False

In [11]:
  # useful for getting number of output classes
folders = glob('/home/avinsah/Desktop/Deep-Learning-Car-Brand-master/Datasets/Train/*')

In [12]:
folders

['/home/avinsah/Desktop/Deep-Learning-Car-Brand-master/Datasets/Train/mercedes',
 '/home/avinsah/Desktop/Deep-Learning-Car-Brand-master/Datasets/Train/audi',
 '/home/avinsah/Desktop/Deep-Learning-Car-Brand-master/Datasets/Train/lamborghini']

In [13]:
# our layers - you can add more if you want
x = Flatten()(resnet.output)

In [14]:
prediction = Dense(len(folders),name = 'output', activation='softmax')(x)

# create a model object
model = Model(inputs=resnet.input, outputs=prediction)

In [15]:
# view the structure of the model
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [16]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [17]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [18]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('Datasets/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 64 images belonging to 3 classes.


In [19]:
test_set = test_datagen.flow_from_directory('Datasets/Test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 58 images belonging to 3 classes.


In [20]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps= len(test_set)
)

/tmp/ipykernel_8444/2916633412.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/50


2022-10-19 11:52:36.102911: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 19267584 exceeds 10% of free system memory.
2022-10-19 11:52:39.042732: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 19267584 exceeds 10% of free system memory.
2022-10-19 11:52:39.052844: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 20313600 exceeds 10% of free system memory.
2022-10-19 11:52:39.059343: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 102760448 exceeds 10% of free system memory.
2022-10-19 11:52:39.154311: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 102760448 exceeds 10% of free system memory.


2/2 [==============================] - 11s 6s/step - loss: 8.2655 - accuracy: 0.3281 - val_loss: 9.3673 - val_accuracy: 0.1552
Epoch 2/50
2/2 [==============================] - 7s 5s/step - loss: 6.0953 - accuracy: 0.3125 - val_loss: 2.5732 - val_accuracy: 0.3966
Epoch 3/50
2/2 [==============================] - 8s 6s/step - loss: 5.7839 - accuracy: 0.3281 - val_loss: 4.4714 - val_accuracy: 0.5172
Epoch 4/50
2/2 [==============================] - 8s 6s/step - loss: 5.2775 - accuracy: 0.4375 - val_loss: 3.6775 - val_accuracy: 0.3276
Epoch 5/50
2/2 [==============================] - 13s 9s/step - loss: 3.0620 - accuracy: 0.4375 - val_loss: 6.4152 - val_accuracy: 0.1552
Epoch 6/50
2/2 [==============================] - 8s 5s/step - loss: 5.0184 - accuracy: 0.4062 - val_loss: 5.9437 - val_accuracy: 0.1552
Epoch 7/50
2/2 [==============================] - 8s 6s/step - loss: 2.7708 - accuracy: 0.4219 - val_loss: 1.3927 - val_accuracy: 0.5345
Epoch 8/50
2/2 [==============================] - 

In [ ]:
# save it as a h5 file
from tensorflow.keras.models import load_model
model.save('model_resnet50.h5')

In [ ]:
y_pred = model.predict(test_set)

2/2 [==============================] - 1s 125ms/step


In [ ]:
y_pred

array([[1.33750867e-02, 7.21894562e-01, 2.64730394e-01],
       [1.14383183e-01, 4.96639043e-01, 3.88977736e-01],
       [2.14545935e-01, 3.77342522e-01, 4.08111602e-01],
       [4.18953551e-03, 9.40512955e-01, 5.52975982e-02],
       [8.97352770e-03, 6.56811059e-01, 3.34215432e-01],
       [3.41001130e-03, 9.53195870e-01, 4.33940552e-02],
       [2.39108764e-02, 2.88243406e-03, 9.73206758e-01],
       [8.39649409e-04, 9.47524846e-01, 5.16355038e-02],
       [2.51902966e-03, 3.95794362e-02, 9.57901537e-01],
       [1.27368541e-02, 5.43374598e-01, 4.43888634e-01],
       [2.17896746e-03, 9.40022647e-01, 5.77984042e-02],
       [8.28564771e-06, 3.85465391e-04, 9.99606311e-01],
       [3.39440778e-02, 1.38749359e-02, 9.52180982e-01],
       [1.36448944e-03, 9.81228292e-01, 1.74072050e-02],
       [9.40844230e-03, 3.83274001e-03, 9.86758769e-01],
       [2.48016720e-03, 4.24767584e-01, 5.72752237e-01],
       [2.18676496e-02, 1.61846936e-01, 8.16285431e-01],
       [2.52958722e-02, 2.09709

In [ ]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
y_pred

array([1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 1, 1, 2, 2,
       1, 1, 1, 0, 2, 1, 2, 1, 2, 0, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 0, 2,
       0, 1, 1, 2, 2, 2, 2, 1, 0, 0, 2, 1, 1, 1])

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
model=load_model('model_resnet50.h5')

In [ ]:
img=image.load_img('Datasets/Test/lamborghini/11.jpg',target_size=(224,224))

In [ ]:
x=image.img_to_array(img)
x

array([[[252., 252., 252.],
        [252., 252., 252.],
        [252., 252., 252.],
        ...,
        [196., 187., 172.],
        [217., 208., 193.],
        [243., 234., 219.]],

       [[252., 252., 252.],
        [252., 252., 252.],
        [252., 252., 252.],
        ...,
        [245., 245., 237.],
        [243., 243., 235.],
        [242., 242., 234.]],

       [[252., 252., 252.],
        [252., 252., 252.],
        [252., 252., 252.],
        ...,
        [240., 249., 248.],
        [242., 251., 250.],
        [242., 251., 250.]],

       ...,

       [[189., 207., 229.],
        [190., 206., 229.],
        [190., 206., 229.],
        ...,
        [171., 180., 187.],
        [171., 180., 187.],
        [171., 180., 187.]],

       [[185., 206., 227.],
        [185., 206., 227.],
        [185., 206., 227.],
        ...,
        [171., 180., 187.],
        [171., 180., 187.],
        [171., 180., 187.]],

       [[185., 206., 227.],
        [185., 206., 227.],
        [185., 2

In [ ]:
x.shape

(224, 224, 3)

In [ ]:
x=x/255

In [ ]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [ ]:
model.predict(img_data)

1/1 [==============================] - 1s 1s/step


array([[0.00120706, 0.00331368, 0.9954792 ]], dtype=float32)

In [ ]:
a=np.argmax(model.predict(img_data), axis=1)

1/1 [==============================] - 0s 22ms/step


In [ ]:
a==1

array([False])

In [ ]:
model.evaluate(test_set)

2/2 [==============================] - 1s 115ms/step - loss: 0.8948 - accuracy: 0.6724


[0.8947731256484985, 0.6724137663841248]